## Reading the articles, token by token. 
(at least the characters match)

# Read in tokens (not necessary)
      # not necessary: token_list = ltf_to_sent.load_ltf(ltf_str)

In [ ]:
from utils import ltf_to_sent
import tqdm
import csv
import os
import spacy
import tqdm
nlp = spacy.load("en_core_web_trf")

def get_raw_text_from_file(filepath):
    basepath = '/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/data/training/data/ltf/'
    filepath = os.path.join(basepath, filepath)
    text_file = open(filepath,"r")
    ltf_str = text_file.read()
    text = ltf_to_sent.ltf_to_plaintext(ltf_str)
    return text
      
def get_gold_labels():
  gold_labels_path = '/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/data/training/annotations/edl.tab'
  lines = []
  with open(gold_labels_path) as tsv:
    for line in csv.reader(tsv, dialect="excel-tab"): #You can also use delimiter="\t" rather than giving a dialect.
      lines.append(line)
  print("example lline:", lines[0])
  return lines

def main():
  labels = get_gold_labels()
  
  # init variables
  first_doc_name = labels[0][3].split(':')[0]
  last_doc_filename = first_doc_name
  count = 0 
  doc = nlp(get_raw_text_from_file(first_doc_name))
  db = spacy.DocBin()
  
  # iterate thru all labels, saving docs as we go
  for _, num, span_text, doc_name, ent_gold, annot_list, NAM_type, const in tqdm.tqdm(labels):
    curr_doc_filename = doc_name.split(':')[0]
    if curr_doc_filename != last_doc_filename:
      # save the last spacy.doc, and load in the next spacy.doc. 
      last_doc_filename = curr_doc_filename
      db.add(doc)
      doc = nlp(get_raw_text_from_file(curr_doc_filename))
    
    # iterate thru the labels and set the spans.
    start_char, end_char = doc_name.split(':')[1].split('-')[0], doc_name.split(':')[1].split('-')[1]
    span = doc.char_span(start_char, end_char+1, label=ent_gold, alignment_mode="contract")
    doc.ents = list(doc.ents) + [span]
    
    print("text:", span_text)
    print("ent_gold:", ent_gold)
    print(doc_name)
    print(curr_doc_filename)
    print(start_char, end_char)
    
    if count > 3:
      break
    count += 1

main()


: 

In [ ]:
""" DEPRICATED::  THIS IS THE BEST WAY TO READ THE LTF FILES. """

# list all files in the directory
basepath = '/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/data/training/data/ltf/'
import os
for file in os.listdir("/home/rohit/Downloads/ltf_files/"):
  if file.endswith(".ltf.xml"):
    filepath = os.path.join(basepath, file)
    text_file = open(filepath,"r")
    ltf_str = text_file.read()
    
    ### Read in plain text
    text = ltf_to_sent.ltf_to_plaintext(ltf_str)
    start_token = 1009
    end_token = 1013
    print(text[start_token : end_token+1]) # NOTE the +1

In [15]:
"""
# Read in tokens (not necessary)
token_list = ltf_to_sent.load_ltf(ltf_str)
"""

'\n# Read in tokens (not necessary)\ntoken_list = ltf_to_sent.load_ltf(ltf_str)\n'

In [ ]:
""" Example code """

db = DocBin() # create a DocBin object

for text, annot in tqdm.tqdm(train): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)


In [ ]:
db.to_disk("./train.spacy") # save the docbin object

### STRETCH GOAL NOT REQUIRED: Parse mention.tab

We DON'T do localization. So just iterate thru the test set and classify each token in the `mention.tab` file into one of the `ontology_types.txt` classes. 

```
mention_path = "/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/kas_source/mentions.tab"

ontology_types_path = "/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/kas_source/utils/rufes/tools/rufes-evaluation/example/data/ontology_types.txt"
```

In [ ]:
import csv
# mention type: “NAM” (for name mentions), “NOM” (for nominal mentions), or “PRO” (for pronominal mentions)
# We don'
mention_path = "/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/kas_source/mentions.tab"
lines = []
with open(mention_path) as tsv:
  for line in csv.reader(tsv, dialect="excel-tab"): #You can also use delimiter="\t" rather than giving a dialect.
    lines.append(line)
print("example lline:", lines[0])

# return
# third field
# Field 7: mention type: “NAM” (for name mentions), “NOM” (for nominal mentions), or “PRO” (for pronominal mentions)
# NIST	annotation-0	Farmers	20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2:0-6	U.S. farmers	PER;PER.Farmer	NOM	1.0
count = 0 
for _, num, text, doc_name, ent_gold, annot_list, NAM_type, const in tqdm.tqdm(lines):

# Parse edl.tab (gold training labels)

In [14]:

# Field 7: mention type: “NAM” (for name mentions), “NOM” (for nominal mentions), or “PRO” (for pronominal mentions)
# NIST	annotation-0	Farmers	20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2:0-6	U.S. farmers	PER;PER.Farmer	NOM	1.0
  ''' add ents to doc '''
  # doc = nlp.make_doc(text)
  # ents = []
  # for start, end, label in annot["entities"]:
  #   span = doc.char_span(start, end, label=label, alignment_mode="contract")
  #   if span is None:
  #     print("Skipping entity")
  #   else:
  #     ents.append(span)
  # doc.ents = ents
  # db.add(doc)
  

example lline: ['NIST', 'annotation-0', 'Farmers', '20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2:0-6', 'U.S. farmers', 'PER;PER.Farmer', 'NOM', '1.0']


  0%|          | 4/4727 [00:00<00:06, 757.50it/s]

text: Farmers
ent_gold: U.S. farmers
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2:0-6
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2
0 6
text: U.S.
ent_gold: United States
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2:118-121
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2
118 121
text: farmers
ent_gold: U.S. farmers
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2:123-129
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2
123 129
text: They
ent_gold: U.S. farmers
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2:311-314
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2
311 314
text: Bloomberg
ent_gold: Bloomberg
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2:397-405
20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2
397 405
